# Illustrate the perturbations

You can only run this notebook if you have access to the respective sdf files of the ligands and the mapping file for each edge.

In [1]:
import os
import re
import sys

import numpy as np
import pandas as pd

import rdkit
from rdkit import Chem
from rdkit.Chem import Draw, PandasTools, rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D, IPythonConsole

rdDepictor.SetPreferCoordGen(True)
from IPython.display import SVG
from cairosvg import svg2png
from svgutils import transform as sg

from IPython.core.display import HTML
from scipy.stats import norm

from PLBenchmarks import targets, ligands, edges
from benchmarkpl import load_data, drawing

from tqdm.notebook import tqdm

sys.path.append(os.path.join(os.getcwd(), '..'))
import benchmarkpl
path = benchmarkpl.__path__[0]
targets.set_data_dir(path)

In [2]:
def draw_edges(target):
    d2ds = []
    if os.path.exists('../../../02_benchmark_calculations/'):
        targets.set_data_dir('../../../02_benchmark_calculations/')
    eSet = edges.EdgeSet(target)
    for name, edge in eSet.items():
            df = edge.get_dataframe()
            df['target'] = target
            # check whether image exists
            os.makedirs(os.path.join(path, targets.get_target_dir(target), '09_illustrations'), exist_ok=True)
            file_path = os.path.join(path, targets.get_target_dir(target), '09_illustrations', f'{name}.svg')
            if False:#os.path.exists(file_path):
                with open(file_path, 'r') as file:
                    img = file.read()
            else:
                # visualization
                m1 = Chem.SDMolSupplier(f'{targets.data_path}/{targets.get_target_dir(target)}/02_ligands/lig_{df[0]}/crd/lig_{df[0]}.sdf', removeHs=False)[0]
                m2 = Chem.SDMolSupplier(f'{targets.data_path}/{targets.get_target_dir(target)}/02_ligands/lig_{df[1]}/crd/lig_{df[1]}.sdf', removeHs=False)[0]
                pairs = np.loadtxt(f'{targets.data_path}/{targets.get_target_dir(target)}/03_hybrid/edge_{df[0]}_{df[1]}/water/crd/pairs.dat')
                # decrement pairs to match rdkit counting from 0!
                pairs -= 1
                img = drawing.drawPerturbationInverted(m1, # rdkit molecule 1
                                               m2, # rdkit molecule 2
                                               pairs, # pairs, np array or list of lists
                                               target='', # string with target name
                                               n1=df[0], # name mol 1
                                               n2=df[1], # name  mol 2
                                               text=''# additional text
                                      )

                with open(file_path, 'w') as file:
                    file.write(img)
                
                svg2png(bytestring=img, write_to=os.path.join(path, targets.get_target_dir(target), '09_illustrations', f'{name}.png'))
                
            #df['img'] = drawPerturbation(m1, m2, pairs, target=t, n1=df[0], n2=df[1]).GetDrawingText()
            d2ds.append(img)
    return HTML(''.join(d2ds))


In [3]:
draw_edges('tyk2')

/home/dhahn3/miniconda3/envs/off/lib/python3.7/site-packages/pandas/core/dtypes/cast.py:1638: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values
